### This is to generate the fine-grained visual understanding dataset

In [2]:
from tqdm import tqdm
import random

In [4]:
step1_json_file = "/home/raja/OVOD/VLM_LLM_exps/VLM_COT/gpt-4o-with-cat-list/output/cub200/original/step1_1000.json"
step2_json_file = "/home/raja/OVOD/VLM_LLM_exps/VLM_COT/gpt-4o-with-cat-list/output/cub200/original/step2_1000.json"

In [5]:
import json
with open(step1_json_file, 'r') as f:
    step1_data = json.load(f)

with open(step2_json_file, 'r') as f:
    step2_data = json.load(f)

In [40]:
from cub200_data import get_cub200_data
test_images = 1000
cub_data, class_to_idx, idx_to_class = get_cub200_data()

dataset = {}

count = 0
incorrect_preds = 0
for image_path, label in tqdm(cub_data, total=test_images):
    image_path = str(image_path)
    image_id = image_path.split("/")[-1].split(".")[0]
    print(f"{count}: {image_id}")
    step1_output = step1_data.get(image_id, None)
    gpt_preds = step1_output["gpt_pred"]
    gpt_labels = step1_output["pred_labels"]
    gt_label = step1_output["gt_label"]

    if (not gt_label in gpt_labels) or (len(gpt_labels) == 0):
        count += 1
        continue

    # find the gt cat name
    gt_cat_name = gpt_preds[gpt_labels.index(gt_label)]
    # print(gt_cat_name)

    filtered_labels = sorted([label for label in gpt_labels if label != -1])
    step2_key = '_'.join(map(str, filtered_labels))

    step2_output = step2_data.get(step2_key, None)
    step2_output = list(step2_output.items())
    random.shuffle(step2_output)
    step2_output = dict(step2_output)

    # print(step2_output)

    curr_desc = []
    idx = 0
    answer = -1
    for cat_name, desc in step2_output.items():
        desc = f"A photo of a {cat_name} with distinctive appearnace described as {desc}"
        curr_desc.append(desc)
        if cat_name == gt_cat_name:
            answer = idx
        idx += 1
    
    if (answer == -1 or len(curr_desc) < 3):
        count += 1
        continue
    
    if (gt_label != gpt_labels[0]):
        incorrect_preds += 1

    curr_dataset = {"image_path": image_path, "text_prompts": curr_desc, "answer": answer} # answer is 0 indexed

    dataset[image_id] = curr_dataset
    
    count += 1
    if (count >= test_images):
        break

with open("grounding_data_with_cate_name.json", "w") as f:
    json.dump(dataset, f, indent=4)


100%|█████████▉| 999/1000 [00:00<00:00, 91583.09it/s]

0: American_Redstart_0122_103102
1: Baird_Sparrow_0006_794579
2: Carolina_Wren_0082_186421
3: Ruby_Throated_Hummingbird_0048_57222
4: Bank_Swallow_0016_129791
5: Pine_Grosbeak_0039_38400
6: Ivory_Gull_0002_49788
7: Sooty_Albatross_0051_796374
8: Scarlet_Tanager_0038_138198
9: Red_Cockaded_Woodpecker_0034_794720
10: Bobolink_0057_10051
11: Song_Sparrow_0038_121666
12: Yellow_Billed_Cuckoo_0033_26521
13: Loggerhead_Shrike_0010_104866
14: Horned_Lark_0064_74864
15: Least_Tern_0026_153702
16: Hooded_Merganser_0028_79061
17: Green_Tailed_Towhee_0062_797424
18: Eared_Grebe_0047_34204
19: Nighthawk_0062_84573
20: Mourning_Warbler_0055_166436
21: Least_Tern_0040_153039
22: Parakeet_Auklet_0067_795964
23: Ivory_Gull_0007_49364
24: American_Pipit_0048_100140
25: Cliff_Swallow_0098_133130
26: Louisiana_Waterthrush_0076_795257
27: Eastern_Towhee_0115_22304
28: Bronzed_Cowbird_0020_796237
29: Bewick_Wren_0055_185230
30: Summer_Tanager_0106_139801
31: Cape_Glossy_Starling_0026_129288
32: Cardinal_00

#### This is to create MCQ dataset for RFT training

In [1]:
from tqdm import tqdm
import random

In [28]:
# top5_pred_file = "/home/raja/OVOD/git_files/VLM-COT/outputs/FLOWERS/FLOWERS_step1_baseline_gemini-2.5-flash-lite-preview-06-17_cat_True.json"
top5_pred_file = "/home/raja/OVOD/git_files/VLM-COT/outputs/FLOWERS/FLOWERS_step1_baseline_gemini-2.5-flash-lite-preview-06-17_cat_True.json"
json_data_path = "/home/raja/OVOD/git_files/VLM-COT/data/oxford_flowers/zero_shot/subsample_base_train.json"

In [29]:
import re
import json
import textwrap
def generate_mcq_data(top5_pred_file, json_data_path):
    
    with open(top5_pred_file, 'r') as f:
        top5_data = json.load(f)
    
    with open(json_data_path, 'r') as f:
        json_data = json.load(f)
    
    print(f"Top 5 data length: {len(top5_data)}")
    print(f"JSON data length: {len(json_data)}")
    # assert len(top5_data) == len(json_data), "Top 5 data and JSON data length mismatch"
    
    dataset = []
    count = 0
    for item in tqdm(json_data, total=len(json_data)):
        
        image_path = item["image_path"]
        gt_cat_name = re.search("<answer>(.*?)</answer>", item["solution"]).group(1)

        image_id = image_path.split("/")[-1].split(".")[0]
        # print(f"{count}: {image_id}")
        try:
            curr_data = top5_data[image_id]
            # print(f"curr_data: {curr_data}")
        except Exception as e:
            # print(f"Error processing image_id {image_id}: {e}")
            continue
        gpt_preds = curr_data["gpt_pred"]
        gpt_labels = curr_data["pred_labels"]
        gt_label = curr_data["gt_label"]        

        if gt_label == -1 or (not gt_label in gpt_labels) or (len(gpt_labels) == 0):
            gpt_preds[-1] = gt_cat_name # replace the last option with the ground truth category name if it is not present in the predictions
        
        random.shuffle(gpt_preds)
        
        letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 
                   'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                   'U', 'V', 'W', 'X', 'Y', 'Z']
        answer = -1
        options = "\n".join([f"{letters[i]}. {option}" for i, option in enumerate(gpt_preds)])
        for option in gpt_preds:
            if option.lower() == gt_cat_name.lower():
                answer = letters[gpt_preds.index(option)]
                break
        
        if answer == -1:
            print(f"gt_cat_name {gt_cat_name}, gpt_preds {gpt_preds}")
            count += 1
            continue
        
        data_json = {
                    "image_path": image_path,
                    "problem": textwrap.dedent(f""" This is an image containing a flower. Please find the most likely flower in the image from the below options.
{options}
Please output the letter corresponding to the correct flower name.
Output the thinking process in <think> </think> and final answer in <answer> </answer> tags.The output answer format should be as follows:
<think> ... </think> <answer>option letter</answer>
Please strictly follow the format. """),
                    "solution": f"<answer>{answer}</answer>"
                }
        
        dataset.append(data_json)
    random.shuffle(dataset)
    print(f"Total count of skipped images: {count}")
    return dataset

In [30]:
dataset = generate_mcq_data(top5_pred_file, json_data_path)

Top 5 data length: 1740
JSON data length: 1744


100%|██████████| 1744/1744 [00:00<00:00, 49251.39it/s]

gt_cat_name ruby-lipped cattleya, gpt_preds ['ruby-lipped_cattleya', 'purple_coneflower', 'moon_orchid', 'barbeton_daisy', 'mexican_aster']
Total count of skipped images: 1


In [7]:
print(dataset[0])

{'image_path': '/home/raja/OVOD/git_files/VLM-COT/data/oxford_flowers/jpg/image_01371.jpg', 'problem': ' This is an image containing a flower. Please find the most likely flower in the image from the below options.\nA. english marigold\nB. sweet pea\nC. marigold\nD. mexican aster\nE. petunia\nPlease output the letter corresponding to the correct flower name.\nOutput the thinking process in <think> </think> and final answer in <answer> </answer> tags.The output answer format should be as follows:\n<think> ... </think> <answer>option letter</answer>\nPlease strictly follow the format. ', 'solution': '<answer>E</answer>'}


In [72]:
print(len(dataset))

1740


In [31]:
with open("/home/raja/OVOD/git_files/VLM-COT/data/oxford_flowers/zero_shot_mcq/subsample_base_train.json", "w") as f:
    json.dump(dataset, f, indent=4)